In [13]:
import pathlib
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.utils.utils import file_lines, get_embeddings_from_str

In [3]:
path_to_data = "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/training_data/bills/train.metadata.embeddings.jsonl.all-MiniLM-L6-v2.parquet"

In [7]:
df.head()

,id,summary,topic,subtopic,subjects_top_term,date,tokenized_text,embeddings
0,110-HR-4,Medicare Prescription Drug Price Negotiation A...,Health,Drug Coverage and Cost,Health,2007-01-05,medicare prescription drug price negotiation a...,-0.054190855 0.07438477 0.04872954 -0.05391701...
1,110-HR-118,Amends part D (Voluntary Prescription Drug Ben...,Health,Drug Coverage and Cost,Health,2007-01-04,amends voluntary prescription drug benefit pro...,-0.07286065 0.117670394 0.06422381 -0.12456543...
2,110-HR-205,Medicare Prescription Drug Enrollment Extensio...,Health,Drug Coverage and Cost,Health,2007-01-04,amends title xviii medicare social security ac...,-0.040297806 0.027768554 0.06305037 -0.0400666...
3,110-HR-426,Medicaid Obesity Treatment Act of 2007 - Amend...,Health,Drug Coverage and Cost,Health,2007-01-11,treatment act amends title xix medicaid social...,-0.025736725 0.049286507 0.067163266 0.0128805...
4,110-HR-464,Compassionate Assistance for Rape Emergencies ...,Health,Drug Coverage and Cost,Health,2007-01-12,compassionate assistance rape emergencies act ...,-0.06253695 0.074354336 -0.034669135 0.0064562...


In [15]:
path_to_data = pathlib.Path(path_to_data)
text_data = "tokenized_text"
raw_text_data = "summary"
get_embeddings = True

if path_to_data.suffix == ".parquet":
    df = pd.read_parquet(path_to_data)
elif path_to_data.suffix in [".json", ".jsonl"]:
    df = pd.read_json(path_to_data, lines=True)

train_data = [doc.split() for doc in df[text_data]]

if get_embeddings:
    if "embeddings" not in df.columns:
        print(f"-- -- Embeddings required but not present in data. They will be calculated from the training data...")
        embeddings = None
        raw_text = [doc.split() for doc in df[raw_text_data]]
    else:
        embeddings = get_embeddings_from_str(df)
        raw_text = None
        print(f"-- -- Loaded embeddings from the DataFrame")

-- -- Loaded embeddings from the DataFrame


In [17]:
from bertopic import BERTopic

In [18]:
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [20]:
docs[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [37]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs


In [45]:
topic_model.probabilities_

array([1.        , 0.74342574, 0.36063674, ..., 1.        , 0.60890926,
       1.        ])

In [49]:
document_info = pd.DataFrame({"Document": docs, "Topic": topic_model.topics_})
topic_info = topic_model.get_topic_info().drop("Count", axis=1)
document_info = pd.merge(document_info, topic_info, on="Topic", how="left")
top_n_words = {topic: " - ".join(list(zip(*topic_model.get_topic(topic)))[0]) for topic in set(topic_model.topics_)}
document_info["Top_n_words"] = document_info.Topic.map(top_n_words)
if topic_model.probabilities_ is not None:
    if len(topic_model.probabilities_.shape) == 1:
        document_info["Probability"] = topic_model.probabilities_
    else:
        document_info["Probability"] = [
            max(probs) if topic != -1 else 1 - sum(probs)
            for topic, probs in zip(topic_model.topics_, topic_model.probabilities_)
        ]
repr_docs = [repr_doc for repr_docs in topic_model.representative_docs_.values() for repr_doc in repr_docs]
document_info["Representative_document"] = False
document_info.loc[document_info.Document.isin(repr_docs), "Representative_document"] = True
document_info[document_info["Representative_document"]==True]

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
59,"\nA word of warning, though: Kryptonite also ...",133,133_lock_kryptonite_cable_locks,"[lock, kryptonite, cable, locks, bike, abus, 6...",[: I was posting to Alt.locksmithing about the...,lock - kryptonite - cable - locks - bike - abu...,1.000000,True
89,"\n\nFirst of all, ""ceremonial law"" is an extra...",176,176_sabbath_moses_tablets_commandments,"[sabbath, moses, tablets, commandments, ceremo...",[I have been following this thread on talk.rel...,sabbath - moses - tablets - commandments - cer...,1.000000,True
92,-= PASTORTALK =-\n\n A weekly dialogue with a...,71,71_schools_silence_moment_christian,"[schools, silence, moment, christian, religion...",[\n\n\n\nAnd maybe they do. But without someb...,schools - silence - moment - christian - relig...,1.000000,True
121,[heats of formation deleted]\n\nThe major prob...,66,66_battery_batteries_concrete_discharge,"[battery, batteries, concrete, discharge, temp...",[[heats of formation deleted]\n\nThe major pro...,battery - batteries - concrete - discharge - t...,1.000000,True
240,\n\nActually the reasons you don't see so many...,151,151_cooling_water_steam_towers,"[cooling, water, steam, towers, plants, nuclea...","[\n Actually, fossil fuel plants run hot...",cooling - water - steam - towers - plants - nu...,1.000000,True
...,...,...,...,...,...,...,...,...
18653,"\n\n\nWhat do you mean ""more comfortable putti...",132,132_eye_prk_cornea_rk,"[eye, prk, cornea, rk, vision, glasses, eyebal...",[I love the FAQ. \n\nThe comment about c...,eye - prk - cornea - rk - vision - glasses - e...,0.649123,True
18677,"\n\nSay, you bought your Saturn at $13k, with ...",106,106_dealer_price_saturn_std,"[dealer, price, saturn, std, car, invoice, sal...","[\nWhoa! Watch your terminology. ""Dealer inv...",dealer - price - saturn - std - car - invoice ...,0.909594,True
18686,I may not be the world's greatest expert on ch...,169,169_wasps_chiggers_centipedes_millipedes,"[wasps, chiggers, centipedes, millipedes, poli...",[-*----\n\nThe millipede's around here (Austin...,wasps - chiggers - centipedes - millipedes - p...,0.866893,True
18803,\nYep.\n,2,2_ites_hello_cheek_hi,"[ites, hello, cheek, hi, yep, huh, ken, ignore...","[\nYep.\n, \n \n ...",ites - hello - cheek - hi - yep - huh - ken - ...,0.452031,True


In [25]:
texts = [" ".join(doc) for doc in train_data]

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(texts)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [50]:
topic_model = BERTopic(nr_topics=50)
topics, probs = topic_model.fit_transform(texts, embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [52]:
document_info = pd.DataFrame({"Document": texts, "Topic": topic_model.topics_})
topic_info = topic_model.get_topic_info().drop("Count", axis=1)
document_info = pd.merge(document_info, topic_info, on="Topic", how="left")
top_n_words = {topic: " - ".join(list(zip(*topic_model.get_topic(topic)))[0]) for topic in set(topic_model.topics_)}
document_info["Top_n_words"] = document_info.Topic.map(top_n_words)
if topic_model.probabilities_ is not None:
    if len(topic_model.probabilities_.shape) == 1:
        document_info["Probability"] = topic_model.probabilities_
    else:
        document_info["Probability"] = [
            max(probs) if topic != -1 else 1 - sum(probs)
            for topic, probs in zip(topic_model.topics_, topic_model.probabilities_)
        ]
repr_docs = [repr_doc for repr_docs in topic_model.representative_docs_.values() for repr_doc in repr_docs]
document_info["Representative_document"] = False
document_info.loc[document_info.Document.isin(repr_docs), "Representative_document"] = True
document_info[document_info["Representative_document"]==True]
document_info

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,medicare prescription drug price negotiation a...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,1.000000,False
1,amends voluntary prescription drug benefit pro...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,1.000000,False
2,amends title xviii medicare social security ac...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,0.667588,False
3,treatment act amends title xix medicaid social...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,0.480859,False
4,compassionate assistance rape emergencies act ...,-1,-1_act_secretary_requires_federal,"[act, secretary, requires, federal, amends, st...",[heroes earning assistance relief tax act inte...,act - secretary - requires - federal - amends ...,0.000000,False
...,...,...,...,...,...,...,...,...
32656,act founding reserve integral combat training ...,5,5_veterans_veteran_care_military,"[veterans, veteran, care, military, service, m...",[measure amended conference report filed house...,veterans - veteran - care - military - service...,1.000000,False
32657,bill directs provide individuals affected brea...,14,14_information_firearms_firearm_person,"[information, firearms, firearm, person, crimi...",[sec requires federal_trade_commission ftc pro...,information - firearms - firearm - person - cr...,0.757476,False
32658,bill bars taking effect month period increases...,2,2_land_water_lands_tribe,"[land, water, lands, tribe, federal, secretary...",[oregon treasures act sec authorizes certain l...,land - water - lands - tribe - federal - secre...,0.783931,False
32659,measure amended introduced summary expanded ac...,34,34_usps_postal_mail_delivery,"[usps, postal, mail, delivery, post, prc, post...",[measure amended introduced summary expanded a...,usps - postal - mail - delivery - post - prc -...,1.000000,True


In [67]:
topic_model.get_document_info(texts)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,medicare prescription drug price negotiation a...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,1.000000,False
1,amends voluntary prescription drug benefit pro...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,1.000000,False
2,amends title xviii medicare social security ac...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,0.667588,False
3,treatment act amends title xix medicaid social...,1,1_health_medicare_care_coverage,"[health, medicare, care, coverage, services, s...",[preserving patient access primary care act am...,health - medicare - care - coverage - services...,0.480859,False
4,compassionate assistance rape emergencies act ...,-1,-1_act_secretary_requires_federal,"[act, secretary, requires, federal, amends, st...",[heroes earning assistance relief tax act inte...,act - secretary - requires - federal - amends ...,0.000000,False
...,...,...,...,...,...,...,...,...
32656,act founding reserve integral combat training ...,5,5_veterans_veteran_care_military,"[veterans, veteran, care, military, service, m...",[measure amended conference report filed house...,veterans - veteran - care - military - service...,1.000000,False
32657,bill directs provide individuals affected brea...,14,14_information_firearms_firearm_person,"[information, firearms, firearm, person, crimi...",[sec requires federal_trade_commission ftc pro...,information - firearms - firearm - person - cr...,0.757476,False
32658,bill bars taking effect month period increases...,2,2_land_water_lands_tribe,"[land, water, lands, tribe, federal, secretary...",[oregon treasures act sec authorizes certain l...,land - water - lands - tribe - federal - secre...,0.783931,False
32659,measure amended introduced summary expanded ac...,34,34_usps_postal_mail_delivery,"[usps, postal, mail, delivery, post, prc, post...",[measure amended introduced summary expanded a...,usps - postal - mail - delivery - post - prc -...,1.000000,True


In [74]:
num_docs = len(texts)
num_topics = len(document_info.Topic.unique())

thetas = np.zeros((num_docs, num_topics))

for id_doc, doc in document_info.iterrows():
    if doc.Topic == -1:
        id_topic = 0
    else:
        id_topic = doc.Topic + 1
    thetas[id_doc, id_topic] = doc.Probability

thetas

array([[0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.66758787, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])